In [1]:
import pandas as pd
import numpy as np

In [80]:
ap_code = pd.read_csv('data/L_AIRPORT.csv')
ap_id = pd.read_csv('data/L_AIRPORT_ID.csv')
airports = pd.read_csv('data/airports.csv')

In [83]:
ap_code.shape, ap_id.shape

((6444, 2), (6428, 2))

In [84]:
ap_code.head()

,Code,Description
0,01A,"Afognak Lake, AK: Afognak Lake Airport"
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip"
2,04A,"Lik, AK: Lik Mining Camp"
3,05A,"Little Squaw, AK: Little Squaw Airport"
4,06A,"Kizhuyak, AK: Kizhuyak Bay"


In [85]:
ap_id.head()

,Code,Description
0,10001,"Afognak Lake, AK: Afognak Lake Airport"
1,10003,"Granite Mountain, AK: Bear Creek Mining Strip"
2,10004,"Lik, AK: Lik Mining Camp"
3,10005,"Little Squaw, AK: Little Squaw Airport"
4,10006,"Kizhuyak, AK: Kizhuyak Bay"


In [86]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [87]:
#only keeping airport codes that are in the airports table
ap_code = ap_code.merge(airports[['IATA_CODE']],how='inner',left_on='Code',right_on='IATA_CODE')

In [88]:
ap_code.head()

,Code,Description,IATA_CODE
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",ABE
1,ABI,"Abilene, TX: Abilene Regional",ABI
2,ABQ,"Albuquerque, NM: Albuquerque International Sun...",ABQ
3,ABR,"Aberdeen, SD: Aberdeen Regional",ABR
4,ABY,"Albany, GA: Southwest Georgia Regional",ABY


In [89]:
#check if unique
ap_code.shape, len(ap_code['Description'].unique())

((322, 3), 322)

In [90]:
match = ap_code.merge(ap_id,how='inner',on='Description')

In [91]:
match.head()

,Code_x,Description,IATA_CODE,Code_y
0,ABE,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",ABE,10135
1,ABI,"Abilene, TX: Abilene Regional",ABI,10136
2,ABQ,"Albuquerque, NM: Albuquerque International Sun...",ABQ,10140
3,ABR,"Aberdeen, SD: Aberdeen Regional",ABR,10141
4,ABY,"Albany, GA: Southwest Georgia Regional",ABY,10146


In [17]:
#Read flight data for the year 2015
data = pd.read_csv(r'data/flights.csv')

/Users/pear/anaconda/envs/env_gdal/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
types = np.asarray([type(x) for x in data['ORIGIN_AIRPORT']])

In [38]:
ind = np.where(types==int)

In [101]:
data[['ORIGIN_AIRPORT']].iloc[ind].merge(
    match,how='left',left_on='ORIGIN_AIRPORT',right_on='Code_y').head()

,ORIGIN_AIRPORT,Code_x,Description,IATA_CODE,Code_y
0,11267,DAY,"Dayton, OH: James M Cox/Dayton International",DAY,11267
1,12266,IAH,"Houston, TX: George Bush Intercontinental/Houston",IAH,12266
2,12266,IAH,"Houston, TX: George Bush Intercontinental/Houston",IAH,12266
3,10821,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",BWI,10821
4,14771,SFO,"San Francisco, CA: San Francisco International",SFO,14771


In [105]:
new_code = data[['ORIGIN_AIRPORT']].iloc[ind].merge(
    match,how='left',left_on='ORIGIN_AIRPORT',right_on='Code_y')['Code_x'].values

In [112]:
#replace the original column with new values
new_code_column = data['ORIGIN_AIRPORT'].values
new_code_column[ind] = new_code
data['ORIGIN_AIRPORT'] = new_code_column

In [118]:
#do the same for destination airport
col = 'DESTINATION_AIRPORT'
types = np.asarray([type(x) for x in data[col]])
ind = np.where(types==int)
new_code = data[[col]].iloc[ind].merge(
    match,how='left',left_on=col,right_on='Code_y')['Code_x'].values
new_code_column = data[col].values
new_code_column[ind] = new_code
data[col] = new_code_column

In [120]:
#write the corrected dataframe to a csv file
data.to_csv('flights_corrected.csv')